In [4]:
"""
下载 PASCAL VOC 2007 数据集并转换为 YOLO 格式,生成 voc.yaml 文件
PASCAL VOC 2007 数据集是计算机视觉领域中一个著名的标准数据集，主要用于目标检测、图像分类和语义分割等任务。
该数据集包含 9963 张图片，分为训练集（5011 张）和测试集（4952 张），
涵盖 20 个类别，如飞机、自行车、鸟、船、瓶子、公共汽车、汽车、猫、椅子、牛、餐桌、狗、马、摩托车、人、盆栽、羊、沙发、火车和电视显示器。
其标注信息以 XML 格式存储，包含目标的边界框、类别标签等。该数据集是许多经典计算机视觉模型的训练和评估基准。
"""
import os
import requests
import tarfile
from lxml import etree
import shutil

# ----------------------
# 配置参数
# ----------------------
VOC_URL = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar"
VOC_TEST_URL = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar"
DATA_DIR = "./dataset/"

# ----------------------
# 下载并解压数据集
# ----------------------
def download_and_extract(url, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir, exist_ok=True)

    filename = os.path.join(dest_dir, url.split("/")[-1])

    # 下载文件
    if not os.path.exists(filename):
        print(f"Downloading {url}...")
        response = requests.get(url, stream=True)
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

    # 解压文件
    print(f"Extracting {filename}...")
    with tarfile.open(filename) as tar:
        tar.extractall(path=dest_dir)



# 转换VOC格式到YOLO格式
# ----------------------
def convert_voc_to_yolo(voc_dir, output_dir):
    print(f"转换VOC格式到YOLO格式: {output_dir}")

    classes = [
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
        'bus', 'car', 'cat', 'chair', 'cow',
        'diningtable', 'dog', 'horse', 'motorbike', 'person',
        'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
    ]

    # 创建训练和测试子目录
    for split in ["train", "test"]:
        os.makedirs(os.path.join(output_dir, f"images/{split}"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, f"labels/{split}"), exist_ok=True)

    # 对 trainval 和 test 分别处理
    for split in ["trainval", "test"]:
        split_type = "train" if split == "trainval" else "test"
        list_path = os.path.join(voc_dir, f"VOCdevkit/VOC2007/ImageSets/Main/{split}.txt")
        if not os.path.exists(list_path):
            print(f"划分文件不存在: {list_path}")
            continue

        with open(list_path) as f:
            ids = [line.strip() for line in f.readlines()]
        print(f"开始处理 {split_type} 集，共 {len(ids)} 张图片")

        for img_id in ids:
            ann_path = os.path.join(voc_dir, f"VOCdevkit/VOC2007/Annotations/{img_id}.xml")
            if not os.path.exists(ann_path):
                print(f"标注文件不存在: {ann_path}")
                continue

            tree = etree.parse(ann_path)
            root = tree.getroot()

            size = root.find("size")
            width = int(size.find("width").text)
            height = int(size.find("height").text)

            yolo_ann = []
            for obj in root.iter("object"):
                cls = obj.find("name").text
                if cls not in classes:
                    continue
                cls_id = classes.index(cls)

                bbox = obj.find("bndbox")
                xmin = float(bbox.find("xmin").text)
                ymin = float(bbox.find("ymin").text)
                xmax = float(bbox.find("xmax").text)
                ymax = float(bbox.find("ymax").text)

                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                yolo_ann.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

            # 保存 YOLO 标签
            label_out_path = os.path.join(output_dir, f"labels/{split_type}/{img_id}.txt")
            with open(label_out_path, "w") as f:
                f.write("\n".join(yolo_ann))

            # 拷贝图像
            src = os.path.join(voc_dir, f"VOCdevkit/VOC2007/JPEGImages/{img_id}.jpg")
            dst = os.path.join(output_dir, f"images/{split_type}/{img_id}.jpg")
            if not os.path.exists(dst):
                try:
                    shutil.copy2(src, dst)
                    # os.symlink(src, dst)  # 尝试创建符号链接
                except:
                    print(f"复制图像失败: {src}")

    # 创建 voc.yaml 文件
    with open("voc.yaml", "w") as f:
        names_str = '\n'.join([f"  {i}: {name}" for i, name in enumerate(classes)])
        f.write(
f"""path: {os.path.abspath(output_dir)}
train: images/train
val: images/train
test: images/test
names:
{names_str}
"""
        )
    print("✅ VOC 转 YOLO 完成，生成 voc.yaml")


# # 下载并解压训练集和测试集
download_and_extract(VOC_URL, DATA_DIR)
download_and_extract(VOC_TEST_URL, DATA_DIR)
# # 执行格式转换 voc->yolo
convert_voc_to_yolo(DATA_DIR, DATA_DIR+"/output")

Extracting ./dataset/VOCtrainval_06-Nov-2007.tar...
Extracting ./dataset/VOCtest_06-Nov-2007.tar...
转换VOC格式到YOLO格式: ./dataset//output
开始处理 train 集，共 5011 张图片
开始处理 test 集，共 4952 张图片
✅ VOC 转 YOLO 完成，生成 voc.yaml


In [5]:
"""
基于YOLO V8的PASCAL VOC 2007的目标检测任务
"""
!pip install ultralytics
# 使用预训练模型
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
device = 'cuda'  # 使用GPU训练,可选cuda或cpu


In [6]:
!nvidia-smi
# 加载预训练模型
model = YOLO("baseModel/yolov8n.pt")  # 使用预训练模型
print("模型加载完成") if model else print("模型加载失败")

Sat Apr 19 03:27:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 训练模型
# 训练参数配置
# model.train(
#     data='voc.yaml',
#     epochs=30,  # 训练轮数
#     batch=64,   # 批处理大小
#     imgsz=640,  # 输入图像大小
#     device=0 if device == "cuda" else "cpu",
#     optimizer='AdamW',  # 优化器
#     lr0=0.0001, # 初始学习率,可选0.001、0.01、0.1等
#     augment=True,
#     # resume=True,  # 中断继续训练
#     # multi_scale=True, 随机改变输入的大小
#     save=True, # 是否保存模型,可选True或False
#     exist_ok=True # 是否覆盖已有模型,可选True或False

# )
model.train(
    data='voc.yaml',
    epochs=50,              # 增加训练轮数
    batch=32,
    imgsz=640,
    device=0,
    optimizer='SGD',        # 改用更适合目标检测的SGD优化器
    lr0=0.001,               # 适当提高初始学习率
    lrf=0.1,                # 添加余弦退火最终学习率
    cos_lr=True,            # 启用余弦学习率调度
    momentum=0.937,           # 调整动量参数
    weight_decay=0.0005,
    warmup_epochs=3.0,      # 适当延长预热
    augment=True,
    hsv_h=0.015,            # 增强颜色空间变换
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,           # 增加旋转角度范围
    translate=0.2,          # 增加平移幅度
    scale=0.5,              # 增大缩放幅度
    shear=0.5,              # 增加剪切幅度
    perspective=0.001,      # 添加透视变换
    flipud=0.2,             # 增加上下翻转概率
    fliplr=0.5,
    mosaic=0.8,             # 保持mosaic增强
    mixup=0.1,              # 添加mixup增强
    copy_paste=0.1,         # 添加copy-paste增强
    close_mosaic=15,        # 延迟关闭mosaic
    multi_scale=True,       # 启用多尺度训练
    erasing=0.5,            # 增加随机擦除概率
    auto_augment='randaugment', # 使用更强数据增强
    nbs=128,                # 增加nominal batch size
    box=5.0,                # 调整损失函数权重
    cls=0.3,
    dfl=1.3,
    save=True,
    exist_ok=True,
    amp=True,               # 保持混合精度训练
    pretrained=True,        # 确保使用预训练权重
    resume=False,
)



Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=baseModel/yolov8n.pt, data=voc.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=15, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=tor

train: Scanning /content/dataset/output/labels/train.cache... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 770.6±656.2 MB/s, size: 73.4 KB)


val: Scanning /content/dataset/output/labels/train.cache... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      10.8G     0.9625      1.393      1.333        206        384:  54%|█████▍    | 85/157 [01:00<00:36,  2.00it/s]

In [ ]:
model.train(
    data='voc.yaml',
    epochs=70,           # 原来训练了40轮，这里设置为总轮数50（=40+10）
    resume=True          # 启用从上次训练中断处继续训练（自动加载runs/train/exp/weights/last.pt）
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 预测输出
import os
import cv2
import torch
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO
import matplotlib.pyplot as plt

# ------------ 全局配置 ------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "runs/detect/train/weights/best.pt"
model = YOLO(MODEL_PATH)
INPUT_PATH = "dataset/output/images/test/"  # 输入路径,可以为图片,视频,文件夹,摄像头编号
# INPUT_PATH = "dataset/output/video/test.mp4"  # 输入路径,可以为图片,视频,文件夹,摄像头编号
# INPUT_PATH=0

SAVE = True  # 是否保存预测结果
OUTPUT_PATH = "predict/"  # 预测结果保存路径

# ------------ 工具函数 ------------
def draw_boxes_pil(image, results):
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()

    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cls_id = int(box.cls)
        conf = float(box.conf)
        label = f"{model.names[cls_id]} {conf:.2f}"

        text_bbox = font.getbbox(label)
        text_w, text_h = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.rectangle([x1, y1 - text_h, x1 + text_w, y1], fill="red")
        draw.text((x1, y1 - text_h), label, fill="white", font=font)

    return image

def save_image(image, save_path, origin_path=None):
    if os.path.isdir(save_path):
        filename = os.path.basename(origin_path)
        save_path = os.path.join(save_path, filename)
    else:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
    image.save(save_path)
    print(f"✅ 已保存图片: {save_path}")

# ------------ 单图预测 ------------
def predict_image(image_path, save=False, save_path=None):
    image = Image.open(image_path).convert("RGB")
    results = model.predict(image_path, imgsz=640, device=DEVICE)
    image = draw_boxes_pil(image, results)

    plt.imshow(image)
    plt.axis("off")
    plt.title("预测结果")
    plt.show()

    if save and save_path:
        save_image(image, save_path, origin_path=image_path)

# ------------ 视频预测 ------------
def predict_video(video_path, save=False, save_path=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ 视频文件无法打开")
        return

    if save:
        if os.path.isdir(save_path):
            filename = os.path.basename(video_path)
            save_path = os.path.join(save_path, f"{os.path.splitext(filename)[0]}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        fps, w, h = cap.get(5), int(cap.get(3)), int(cap.get(4))
        out = cv2.VideoWriter(save_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, imgsz=640, device=DEVICE)
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls_id = int(box.cls)
            conf = float(box.conf)
            label = f"{model.names[cls_id]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("预测中 - 按 Q 退出", frame)
        if save:
            out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    if save:
        out.release()
        print(f"✅ 已保存视频: {save_path}")
    cv2.destroyAllWindows()

# ------------ 文件夹批量图片 ------------
def predict_folder(folder_path, save=False, output_dir=None):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                img_path = os.path.join(root, file)
                image = Image.open(img_path).convert("RGB")
                results = model.predict(img_path, imgsz=640, device=DEVICE)
                image = draw_boxes_pil(image, results)

                if save and output_dir:
                    rel_path = os.path.relpath(img_path, folder_path)
                    save_path = os.path.join(output_dir, rel_path)
                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                    image.save(save_path)

    if save:
        print(f"✅ 文件夹预测完成，结果已保存至: {output_dir}")

# ------------ 摄像头实时预测 ------------
def predict_camera(index=0):
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        print(f"❌ 无法打开摄像头 {index}")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, imgsz=640, device=DEVICE)
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls_id = int(box.cls)
            conf = float(box.conf)
            label = f"{model.names[cls_id]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("摄像头预测 - 按 Q 退出", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# ------------ 总入口函数 ------------
def run_predict(path, save=False, save_path=None):
    if isinstance(path, int):
        predict_camera(index=path)
    elif os.path.isfile(path):
        ext = os.path.splitext(path)[1].lower()
        if ext in [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]:
            predict_image(path, save, save_path)
        elif ext in [".mp4", ".avi", ".mov", ".mkv"]:
            predict_video(path, save, save_path)
    elif os.path.isdir(path):
        predict_folder(path, save, save_path)
    else:
        print("❌ 无效路径，请确认输入正确的图片/视频/文件夹/摄像头编号")

# ------------ 示例调用 ------------
run_predict(INPUT_PATH, SAVE, OUTPUT_PATH)      # 预测输出
